In [1]:
import sys
import pathlib
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
import plotly.express as px
from pycytominer.cyto_utils import infer_cp_features
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import pdfkit
import pyarrow as pa
import pyarrow.parquet as pq

# import Union
from typing import Union

sys.path.append("..")
# from ..utils.utils import df_stats
import matplotlib.pyplot as plt

In [2]:
# Define inputs
feature_file = pathlib.Path(
    "../../Extracted_Features_(CSV_files)/SHSY5Y_run_sc_norm.parquet"
)
feature_df = pq.read_table(feature_file).to_pandas()
# feature_df = pd.read_csv(feature_file, engine="pyarrow")

In [3]:
# define output file path
feature_df_out_path = pathlib.Path(
    "../../Extracted_Features_(CSV_files)/feature_df_sc_norm.parquet"
)

In [4]:
print(feature_df.shape)
feature_df.head()

(597902, 2926)


,Metadata_cell_type,Metadata_Well,Metadata_number_of_singlecells,Metadata_incubation inducer (h),Metadata_inhibitor,Metadata_inhibitor_concentration,Metadata_inhibitor_concentration_unit,Metadata_inducer1,Metadata_inducer1_concentration,Metadata_inducer1_concentration_unit,...,Nuclei_Texture_Variance_CorrGasdermin_3_02_256,Nuclei_Texture_Variance_CorrGasdermin_3_03_256,Nuclei_Texture_Variance_CorrMito_3_00_256,Nuclei_Texture_Variance_CorrMito_3_01_256,Nuclei_Texture_Variance_CorrMito_3_02_256,Nuclei_Texture_Variance_CorrMito_3_03_256,Nuclei_Texture_Variance_CorrPM_3_00_256,Nuclei_Texture_Variance_CorrPM_3_01_256,Nuclei_Texture_Variance_CorrPM_3_02_256,Nuclei_Texture_Variance_CorrPM_3_03_256
0,SH-SY5Y,B13,3765,6,Media ctr,NaN,None,media ctr,None,None,...,0.011881,0.003817,0.353469,0.365618,0.378125,0.342100,0.099225,0.096054,0.110432,0.107764
1,SH-SY5Y,B13,3765,6,Media ctr,NaN,None,media ctr,None,None,...,-0.044113,-0.042322,1.780397,1.603914,1.571446,1.582787,0.020683,0.016122,0.012990,0.013631
2,SH-SY5Y,B13,3765,6,Media ctr,NaN,None,media ctr,None,None,...,-0.051521,-0.054760,0.019005,0.041005,0.029673,0.007339,0.027519,0.045982,0.037653,0.017281
3,SH-SY5Y,B13,3765,6,Media ctr,NaN,None,media ctr,None,None,...,-0.058159,-0.059141,-0.005212,-0.010673,0.000258,0.003475,-0.002945,0.000293,0.008048,-0.002151
4,SH-SY5Y,B13,3765,6,Media ctr,NaN,None,media ctr,None,None,...,-0.034966,-0.030255,-0.044078,-0.052547,-0.042483,-0.029738,0.016846,0.006499,0.014114,0.026206


In [5]:
# removing costes features as they behave with great variance across all data
# feature_df = feature_df.drop(feature_df.filter(regex="Costes").columns, axis=1)
# print(feature_df.shape)
# feature_df

In [6]:
# replacing '/' in treatment dosage column to avoid errors in file interpolation including such strings
feature_df = feature_df.replace(to_replace="/", value="_per_", regex=True)

In [7]:
# Recycled code from: https://github.com/WayScience/NF1_SchwannCell_data/blob/main/5_analyze_data/notebooks/linear_model/fit_linear_model.ipynb
cell_count_df = (
    feature_df.groupby("Metadata_Well")["Metadata_Plate"]
    .count()
    .reset_index()
    .rename(columns={"Metadata_Plate": "Metadata_number_of_singlecells"})
)

In [8]:
feature_df.head()

,Metadata_cell_type,Metadata_Well,Metadata_number_of_singlecells,Metadata_incubation inducer (h),Metadata_inhibitor,Metadata_inhibitor_concentration,Metadata_inhibitor_concentration_unit,Metadata_inducer1,Metadata_inducer1_concentration,Metadata_inducer1_concentration_unit,...,Nuclei_Texture_Variance_CorrGasdermin_3_02_256,Nuclei_Texture_Variance_CorrGasdermin_3_03_256,Nuclei_Texture_Variance_CorrMito_3_00_256,Nuclei_Texture_Variance_CorrMito_3_01_256,Nuclei_Texture_Variance_CorrMito_3_02_256,Nuclei_Texture_Variance_CorrMito_3_03_256,Nuclei_Texture_Variance_CorrPM_3_00_256,Nuclei_Texture_Variance_CorrPM_3_01_256,Nuclei_Texture_Variance_CorrPM_3_02_256,Nuclei_Texture_Variance_CorrPM_3_03_256
0,SH-SY5Y,B13,3765,6,Media ctr,NaN,None,media ctr,None,None,...,0.011881,0.003817,0.353469,0.365618,0.378125,0.342100,0.099225,0.096054,0.110432,0.107764
1,SH-SY5Y,B13,3765,6,Media ctr,NaN,None,media ctr,None,None,...,-0.044113,-0.042322,1.780397,1.603914,1.571446,1.582787,0.020683,0.016122,0.012990,0.013631
2,SH-SY5Y,B13,3765,6,Media ctr,NaN,None,media ctr,None,None,...,-0.051521,-0.054760,0.019005,0.041005,0.029673,0.007339,0.027519,0.045982,0.037653,0.017281
3,SH-SY5Y,B13,3765,6,Media ctr,NaN,None,media ctr,None,None,...,-0.058159,-0.059141,-0.005212,-0.010673,0.000258,0.003475,-0.002945,0.000293,0.008048,-0.002151
4,SH-SY5Y,B13,3765,6,Media ctr,NaN,None,media ctr,None,None,...,-0.034966,-0.030255,-0.044078,-0.052547,-0.042483,-0.029738,0.016846,0.006499,0.014114,0.026206


In [9]:
# show max column in pandas df
pd.set_option("display.max_columns", 100)

In [10]:
# replace nan values with 0
feature_df["Metadata_inducer1_concentration"] = feature_df[
    "Metadata_inducer1_concentration"
].fillna(0)
feature_df["Metadata_inducer2_concentration"] = feature_df[
    "Metadata_inducer2_concentration"
].fillna(0)
feature_df["Metadata_inhibitor_concentration"] = feature_df[
    "Metadata_inhibitor_concentration"
].fillna(0)

#### Combine Inducer1 and Inducer2 into one column

In [11]:
# treatment column merge
conditions = [
    (feature_df["Metadata_inducer2"].isnull()),
    feature_df["Metadata_inducer2"].notnull(),
]

results = [
    (feature_df["Metadata_inducer1"]).astype(str),
    (feature_df["Metadata_inducer1"] + "_" + feature_df["Metadata_inducer2"]).astype(
        str
    ),
]
feature_df["Metadata_Treatment"] = np.select(conditions, results)

# dose column merge
conditions = [
    (feature_df["Metadata_inducer2"].isnull()),
    feature_df["Metadata_inducer2"].notnull(),
]

results = [
    (feature_df["Metadata_inducer1_concentration"].astype(str)).astype(str),
    (
        feature_df["Metadata_inducer1_concentration"].astype(str)
        + "_"
        + feature_df["Metadata_inducer2_concentration"].astype(str)
    ).astype(str),
]
feature_df["Metadata_Dose"] = np.select(conditions, results)

## 1 Beta Column condition generation

In [12]:
feature_df["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"] = (
    feature_df["Metadata_Treatment"]
    + "_"
    + feature_df["Metadata_Dose"].astype(str)
    + "_"
    + feature_df["Metadata_inhibitor"].astype(str)
    + "_"
    + feature_df["Metadata_inhibitor_concentration"].astype(str)
).astype(str)

feature_df["twob_Metadata_Treatment_Dose_Inhibitor_Dose"] = (
    feature_df["Metadata_Treatment"]
    + "_"
    + feature_df["Metadata_inhibitor"].astype(str)
    + "_"
    + feature_df["Metadata_inhibitor_concentration"].astype(str)
    + "__"
    + feature_df["Metadata_Dose"].astype(str)
).astype(str)

feature_df["threeb_Metadata_Treatment_Dose_Inhibitor_Dose"] = (
    feature_df["Metadata_Treatment"]
    + "__"
    + feature_df["Metadata_Dose"].astype(str)
    + "__"
    + feature_df["Metadata_inhibitor"].astype(str)
    + "_"
    + feature_df["Metadata_inhibitor_concentration"].astype(str)
).astype(str)
feature_df["fourb_Metadata_Treatment_Dose_Inhibitor_Dose"] = (
    feature_df["Metadata_Treatment"]
    + "__"
    + feature_df["Metadata_Dose"].astype(str)
    + "__"
    + feature_df["Metadata_inhibitor"].astype(str)
    + "__"
    + feature_df["Metadata_inhibitor_concentration"].astype(str)
).astype(str)

In [ ]:
feature_df["Metadata_inducer1_concentration"] = pd.to_numeric(
    feature_df["Metadata_inducer1_concentration"]
)

In [ ]:
# feature_df.to_csv(feature_df_out_path, index=False)
feature_df_table = pa.Table.from_pandas(feature_df)
pq.write_table(feature_df_table, feature_df_out_path)